Threshold Paillier Cryptosystem
=============

## A Distributed Decryption for Paillier

The protocol offers a distributive decryption for Paillier with simulation based security against malicious adversaries without randomness extraction. It is comprised of the following two subprotocols:

1. The parties produce shares of a value d similarly to the **Damgard-Jurik scheme**

2. The parties run the distributed decryption algorithm using their shares.

use $g = 1 + N$ as a generator of the subgroup of $Z^∗_{N^2}$ of order $N$. Encryption of a plaintext $m$ with randomness $r$ is then,

$$
Enc_N(m, r) = (1+N)^m \cdot r^N \mod N^2
$$

### Generating a Shared Paillier Decryption Key

We now present our protocol for generating a shared Paillier decryption key. As stated, similarly to **Damgard and Jurik** , we share a decryption exponent as follows:



$$
d \equiv \left\{
\begin{aligned}
0 &\mod \phi(N) \\
1 &\mod N
\end{aligned}
\right\}
$$

#### A distributed generation of a shared Paillier decryption key with passive security:

* **Input**:

A public **RSA** modulus $N=pq$ with unknow factorization, additive shares of $\phi(N)$:

$$
sk_0 = N-p_0-q_0+1\\
sk_1 = -p_1-q_2
$$

$sk_0, sk_1$ hold by $P_0, and P_1$ respectively, A public **ElGamel** key $(g,h)$ with secret key shared between the parities. A public Pailler key $N_0>>N^3$ with the secret key hold by $P_0$

In [1]:
from klefki.numbers.primes import generate_prime
from klefki.numbers import lcm
from klefki.mpc.secret_sharing import additive_share
from klefki.types.algebra.meta import field
import random

In [2]:
P, Q = generate_prime(32), generate_prime(32)
N = P * Q
Lam = (P-1) * (Q-1)


In [3]:
sk0, sk1 = additive_share(Lam, field(N), 2)
sk0, sk1 = sk0.value, sk1.value

In [4]:
from klefki.crypto.elgamal import ElGamal

In [5]:
x = 42
el = ElGamal(x)

In [6]:
g, h = el.pubkey

##### **The  protocol**

1. $P_0$ encrypts $sk_0$ using $N_0$ and sends this to $P_1$

In [7]:
from klefki.crypto.paillier import Paillier

In [8]:
p0, q0 = generate_prime(32**2), generate_prime(32**2)

In [9]:
N0 = p0 * q0

In [10]:
p0 = Paillier(p0, q0)

In [11]:
e_sk0 = p0.E(sk0)

In [12]:
p0.D(e_sk0).value == sk0

True

----------------

2. $P_1$ picks $r_1 \in Z_N$ and $r_\sigma \in Z_{2^{logN+k}}$ uniformly at random (for a statistical parameter $κ$ that enables to mask the secret key). $P_1$ computes an encryption of

$$
(sk_0 + sk_1) \cdot r_1 + N \cdot r_\sigma
$$

using the homomorphic property of Paillier encryption. This is rerandomized and sent to $P_0$ .

In [13]:
from klefki.types.algebra.utils import randfield
from klefki.types.algebra.meta import field
from math import log

In [14]:
r1 = randfield(field(N)).value
ro = randfield(field(int(2 ** log(N, 2)))).value

In [15]:
re = (p0.E(sk1) * e_sk0) ** r1 * p0.E(N) ** ro

In [16]:
p0.D((p0.E(sk1) * e_sk0) ** r1 * p0.E(N) ** ro) == p0.D(p0.E((sk0+sk1) * r1 + N * ro))

True

In [17]:
p0.D((p0.E(sk1) * e_sk0) ** r1 * p0.E(N) ** ro)

N::76383424111849052610663299625242686777

In [18]:
 p0.D(p0.E((sk0+sk1) * r1 + N * ro))

N::76383424111849052610663299625242686777

------------



3. $P_0$ decrypts, thus obtaining plaintext $r_0 ; P_0$ computes $r_0^{−1} \mod N$ and encrypts this as well as plaintext $sk_0 (r_0^{−1} \mod N)$ under public-key $N_0$ . Both ciphertexts are sent to $P_1$

In [19]:
r0 = p0.D(re)

In [20]:
(~field(N)(r0)).value

6423860760600523678

In [21]:
er0 = p0.E((~field(N)(r0)))

In [22]:
esk_r0 = p0.E(sk0 * (~field(N)(r0)).value)

--------------
4. Based on the encryptions of $r_0^{-1} \mod N$ and $sk_0(r_0^{-1} \mod N)$, $P_1$ computes an encryption of:

\begin{align}
d&=(sk_0(r_0^{-1} \mod N) \cdot r_1 + (r_0^{-1} \mod N)(sk_1 \cdot r_1))\\
&=r_1(sk_0+sk_1)(r_0^{01} \mod N)\\
&=r_1 \cdot \phi(N) \cdot (r_0^{-1} \mod N)
\end{align}

$P_1$ then picks $\bar{d}_1$ uniformly at random in $Z_{2^3logN+κ}$ , and computes and rerandomizes an encryption of $d + \bar{d}_1$ . This is sent to $P_0$ and ﬁnally, $P_1$ sets its share of d to the integer $-\bar{d}_1$ .

In [23]:
e_d = esk_r0 ** r1 * (er0 ** sk1) ** r1

In [24]:
p0.D(e_d).value == r1 * (P-1) * (Q-1) * (~field(N)(r0)).value

True

In [25]:
d1_ = randfield(field(int(2 ** (3 * log(N, 2))))).value

e_d0 = p0.E(d1_) * e_d

In [26]:
d1 = -d1_

----------
5. $P_0$ decrypts and obtains $d_0$ : its share of $d$.

In [27]:
d0 = p0.D(e_d0).value

##### Correctness:



In [28]:
d = p0.D(e_d).value

In [29]:
p0.D(e_d).value == d1 + d0

True

In [30]:
assert d % ((P-1) * (Q-1)) == 0
assert d % (P * Q) == 1

### Performing a Joint Paillier Decryption

To perform a joint decryption of some ciphertext $c$, both parties need to raise $c$ to their share of the key, $d_0$ or $d_1$ . They then demonstrate that this has been computed correctly using the commitments of the shares. The plaintext is immediately computable from $c^{d_0}$ and $c^{d_1}$ .

#### A distributed Paillier decryption with a shared key:

**Inputs:** A public Paillier key $N=pq$ with unknown factorization and a ciphertext $C=E_N(m,r)$.

Party $P_i$ hold its share $d_i$ of the secret decryption exponent, $d=d_0+d_1$ where 

$$
d \equiv 1 \mod N \wedge d \equiv 0 \mod \phi(N)
$$

Finall, the parties hold commitments to (or rather: ElGamal encryptions of) their key-shares.

**The protocol:**

1. $P_0$ sends $c_0=c^{sk0} \mod N^2$ to $P_1$. Moreover, $P_0$ demonstrates that this has been done correctly by executing $\pi_{EQ}$, i.e., that the committed number equals the discrete log of c 0 with base c and the plaintext encrypted with ElGamal.

2. $P_1$ sends $c_1=c^{sk2} \mod N^2$ to $P_0$. Moreover, $P_1$ demonstrates that this has been done correctly by executing $\pi_{EQ}$, i.e., that the committed number equals the discrete log of c 0 with base c and the plaintext encrypted with ElGamal.

In [31]:
from klefki.crypto.damgard_jurik import DJPaillier

In [32]:
m = 42
p = DJPaillier(P, Q).privkey

NameError: name 'p' is not defined

In [ ]:
N = P * Q

In [ ]:
p % lcm(P-1, Q-1)


In [ ]:
 p % N

## a protocol for generating a threshold key

Constructing a threshold key essentially consists of computing a Shamir sharing of this. Our solution consists of two steps: 

* 1) First, compute an additive sharing of $d$. 

* 2) Then, compute Shamir shares of this and decrypt these toward the relevant parties.

$$
\phi(N) \cdot (\phi(N)^{-1} \mod N) \equiv \left\{
\begin{aligned}
0 &\mod \phi(N) \\
1 &\mod N
\end{aligned}
\right\}
$$

In [ ]:
from klefki.numbers.primes import generate_prime
from klefki.types.algebra.meta import field
from klefki.numbers import invmod
from klefki.mpc.secret_sharing import additive_share
from functools import reduce
from operator import add

In [ ]:
N = P * Q
phi = (P - 1) * (Q - 1)
inv_phi = invmod(phi, N)

In [ ]:
assert phi * inv_phi % phi == 0
assert phi * inv_phi % N == 1

Ref:

* Carmit Hazay, Gert Læssøe Mikkelsen, etc.., Efﬁcient RSA Key Generation and Threshold Paillier in the Two-Party Setting